In [19]:
! nvidia-smi

Sat May 21 22:47:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.68.02    Driver Version: 512.77       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   43C    P8    12W /  N/A |      0MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [20]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

# Seq2seq

In [21]:
%matplotlib inline

В этом проекте мы будем учиться делать перевод с французского на английский. Примерно так:

    [KEY: > input, = target, < output]

    > il est en train de peindre un tableau .
    = he is painting a picture .
    < he is painting a picture .

    > pourquoi ne pas essayer ce vin delicieux ?
    = why not try that delicious wine ?
    < why not try that delicious wine ?

    > elle n est pas poete mais romanciere .
    = she is not a poet but a novelist .
    < she not not a poet but a novelist .

    > vous etes trop maigre .
    = you re too skinny .
    < you re all alone .

Для этого мы будем исплользовать мощную идею «sequence-to-sequence» сетей (https://arxiv.org/abs/1409.3215), в которых две рекуррентные сети обучаются вместе для преоразования одной последовательности в другую.

* Encoder-сеть сжимает входную последовательность в вектор.
* Decoder-сеть разжимает этот вектор в новую последовательность.

Всё как с автоэнкодерами, только encoder и decoder из разных доменов.

Чтобы вся эта схема обучалась стабильнее, мы будем использовать механизм attention (https://arxiv.org/abs/1409.0473), позволяющий декодеру «фокусироваться» на специфичных токенах входной последовательности.

**Рекомендуемое чтение:**

-  Learning Phrase Representations using RNN Encoder-Decoder for
   Statistical Machine Translation (https://arxiv.org/abs/1406.1078)
-  Sequence to Sequence Learning with Neural
   Networks (https://arxiv.org/abs/1409.3215)
-  Neural Machine Translation by Jointly Learning to Align and
   Translate 9https://arxiv.org/abs/1409.0473)
-  A Neural Conversational Model (https://arxiv.org/abs/1506.05869>)

Если кто-то пропустил предыдущие занатия, то лучше сначала сделать их: основные концепции такие же, как в языковых моделях.

In [22]:
# осторожно: тетрадка старая

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LENGTH = 1000

In [23]:
torch.cuda.is_available()

True

## Данные

В этом проекте мы будем работать с кучей пар предложений на английском и французском.

Скачайте данные отсюда (https://download.pytorch.org/tutorial/data.zip) и возьмите оттуда файлик eng-fra.txt. В нём должно быть много строчек примерно такого формата:

    I am cold.    J'ai froid.

Делать предобработку будем по аналогии с char-level RNN-ками из предыдущих туториалов, только на этот раз нам важно отдельно запариться с EOS (end-of-sequence) — специальным токеном, который сеть будет генерировать при окончании генерации предложения.

In [24]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
    def __repr__(self):
        most_popular_words = sorted(
            self.word2count.keys(), key=lambda word: self.word2count[word], reverse=True
        )[:10]
        most_popular_words = ", ".join(most_popular_words)
        return f"Language: {self.name} | Num words: {self.n_words} | Most popular: {most_popular_words}"

Все файлы в юникоде. Чтобы  облегчить нам работу, мы переведем все в ASCII, сделаем lowercase и выкинем большинство пунктуации.

In [25]:
# "hello!" -> hello, ! 

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# def normalizeString(s):
#     s = unicodeToAscii(s.lower().strip())
#     s = re.sub(r"([.!?])", r" \1", s)
#     s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
#     return s

def normalizeString(text):
    text = text.lower()
    text = re.sub(r'[^\sa-zA-Z0-9@\[\]]',' ',text) # Удаляет пунктцацию
    text = re.sub(r'\w*\d+\w*', '', text) # Удаляет цифры
    text = re.sub('\s{2,}', " ", text) # Удаляет ненужные пробелы
    text = text.replace('\n', ' ')
    return text

При чтении данных разделим файл на строки, а строки на пары.




In [26]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")
    
    import pandas as pd
    df = pd.read_csv('happiness_provokers', encoding='utf-8')

    # Get pairs and normalize
    pairs = list(zip([normalizeString(s) for s in list(df['query'].values)], [normalizeString(s) for s in list(df.reply.values)]))
    
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [9]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [27]:
from transformers import GPT2Tokenizer

In [28]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [29]:
seq = """Hi bruh wtf r u doing ahh?"""

In [30]:
for i in tokenizer.encode(seq):
  print(tokenizer.decode(i))

Hi
 bru
h
 w
tf
 r
 u
 doing
 a
hh
?


Полный процесс такой:

- Считать текстовый файл, просплитить по линиям, а затем по парам.
- Нормализовать текст, профильтровать по длине.
- Сделать готовые списки слов из сырых предложений в каждом языке.

In [31]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    pairs = pairs[::]
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng_user', 'eng_emotion_provoker')
print(random.choice(pairs))

Reading lines...
Read 32885 sentence pairs
Counting words...
Counted words:
eng_user 18346
eng_emotion_provoker 18295
(' no i m not did you think this is most important for everyone ', ' i think everyone should stand up against hate ')


In [32]:
input_lang

Language: eng_user | Num words: 18346 | Most popular: , i, the, that, a, to, it, of, is, in

In [34]:
output_lang

Language: eng_emotion_provoker | Num words: 18295 | Most popular: , i, the, that, a, to, it, of, is, you

In [35]:
len(pairs)

32885

In [36]:
pairs[24]

(' i don t know but i am at the end of knowledge on this subject what do you think about the facts of comics i would have thought us invented them but it s japan or uk ',
 ' japan has a great printing history i am not surprised about this do you read comics books i have read a few but not about super heroes ')

The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A `Sequence to Sequence network <https://arxiv.org/abs/1409.3215>`__, or
seq2seq network, or `Encoder Decoder
network <https://arxiv.org/pdf/1406.1078v3.pdf>`__, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.


Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

Consider the sentence "Je ne suis pas le chat noir" → "I am not the
black cat". Most of the words in the input sentence have a direct
translation in the output sentence, but are in slightly different
orders, e.g. "chat noir" and "black cat". Because of the "ne/pas"
construction there is also one more word in the input sentence. It would
be difficult to produce a correct translation directly from the sequence
of input words.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.




The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.




(https://blog.floydhub.com/content/images/2019/07/image17-1.jpg)

In [37]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        # num_embedding = vocab_size_fra
        self.embedder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        # (batch_size, num_words) -> (batch_size, num_words, dim_1)
        embeddings = self.embedder(input).view(1, 1, -1)
        # (batch_size, num_words, dim_2)
        output, hidden = self.gru(embeddings, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [38]:
tokens = torch.randint(0, 1000, size=(128, 40))
embedder = nn.Embedding(1000, 128)  # здесь лежит матрица

In [39]:
onehot = torch.nn.functional.one_hot(tokens, num_classes=1000)

In [40]:
embeddingds_first_way = embedder(tokens)

In [41]:
embeddingds_first_way.size()

torch.Size([128, 40, 128])

The Decoder
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




Simple Decoder
_________________

In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>`` or `<BOS>`
token, and the first hidden state is the context vector (the encoder's
last hidden state).
The last token is `<EOS>` whats mean end of string




In [42]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size

        self.embedder = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        # (batch_size, num_words, dim)
        # (1, 1, num_words * dim)
        output = self.embedder(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        # (batch_size, num_words, dim) -> (batch_size, num_words, num_classes)
        # (batch_size, num_words, vocab_size_eng)
        output = self.out(output[0])
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

I encourage you to train and observe the results of this model, but to
save space we'll be going straight for the gold and introducing the
Attention Mechanism.




Attention Decoder
____________

If only the context vector is passed betweeen the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to "focus" on a different part of
the encoder's outputs for every step of the decoder's own outputs. First
we calculate a set of *attention weights*. These will be multiplied by
the encoder output vectors to create a weighted combination. The result
(called ``attn_applied`` in the code) should contain information about
that specific part of the input sequence, and thus help the decoder
choose the right output words.

Calculating the attention weights is done with another feed-forward
layer ``attn``, using the decoder's input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.




In [ ]:
# v^TWm
# U^T tanh (W_1 v + W_2 m)

In [43]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(
                torch.cat((embedded[0], hidden[0]), 1)
            ), 
            dim=1
        )
        attn_applied = torch.bmm(
            attn_weights.unsqueeze(0),
            encoder_outputs.unsqueeze(0)
        )

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

<div class="alert alert-info"><h4>Note</h4><p>There are other forms of attention that work around the length
  limitation by using a relative position approach. Read about "local
  attention" in `Effective Approaches to Attention-based Neural Machine
  Translation <https://arxiv.org/abs/1508.04025>`__.</p></div>

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.




In [44]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but `when the trained
network is exploited, it may exhibit
instability <http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf>`__.

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the translation
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [45]:
teacher_forcing_ratio = 0.5


def train(
    input_tensor, 
    target_tensor,
    encoder, 
    decoder, 
    encoder_optimizer,
    decoder_optimizer, 
    criterion,
    max_length=MAX_LENGTH
):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        # y_true: [sos, i, love, pizza, eos]
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            # [0.9, 0.1, 0.0]
            # [1, 0, 0]
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            # beam_search is betters
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                # y_true: [sos, i, eos]
                # [sos, i, eos, love]
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [46]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [47]:
PATH_ENC = "blablaenc.pt"
PATH_DECO = "blabladeco.pt"

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % 100 == 0:
          torch.save(encoder.state_dict(), PATH_ENC)
          torch.save(decoder.state_dict(), PATH_DECO)

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            output_words_check, _ = evaluate(encoder, decoder, 'what are you doing my dear friend')
            print(output_words_check)
            print()

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

Plotting results
----------------

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [48]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [49]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [50]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

Training and Evaluating
=======================

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. After about 40 minutes on a MacBook CPU we'll get some
reasonable results.

.. Note::
   If you run this notebook you can train, interrupt the kernel,
   evaluate, and continue training later. Comment out the lines where the
   encoder and decoder are initialized and run ``trainIters`` again.




In [51]:
hidden_size = 256

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

encoder1.load_state_dict(torch.load(PATH_ENC)) # можно и другую директорию, но вот это прямо внутри вашего гугл диска
attn_decoder1.load_state_dict(torch.load(PATH_DECO))

trainIters(encoder1, attn_decoder1, 75000, print_every=50)

0m 11s (- 292m 55s) (50 0%) 6.3003
['', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 

1m 13s (- 86m 47s) (1050 1%) 4.8336
['', 'i', 'is', 'the', 'i', 'of', 'the', 'of', 'the', '', '<EOS>']

1m 16s (- 85m 56s) (1100 1%) 4.5669
['', 'i', 'don', 't', 'know', 'that', 'is', 'the', '', '<EOS>']

1m 19s (- 85m 2s) (1150 1%) 4.5388
['', 'i', 'would', 'be', 'i', 'think', 'it', 'was', 'it', 'was', 'a', 'i', '', '<EOS>']

1m 22s (- 84m 23s) (1200 1%) 4.9359
['', 'yes', 'you', 'know', 'the', '', '<EOS>']

1m 24s (- 83m 29s) (1250 1%) 4.5608
['', 'i', 'know', 'that', 'i', 'know', 'that', 'of', 'the', 'on', 'the', 'on', 'the', '', '<EOS>']

1m 27s (- 82m 54s) (1300 1%) 4.8308
['', 'that', 's', 'you', 'a', 'a', 'good', 'the', 'the', 'the', '', '<EOS>']

1m 30s (- 82m 33s) (1350 1%) 5.0135
['', 'i', 'i', 'was', 'a', 'i', 'was', 'a', 'i', 'was', 'a', 'i', 'was', 'a', 'i', 'was', 'a', 'i', 'was', 'a', 'i', 'was', 'a', 'i', 'was', 'a', 'i', 'was', 'a', 'i', 'was', 'a', 'i', 'was', 'a', 'i', '', '<EOS>']

1m 33s (- 82m 7s) (1400 1%) 4.7195
['', 'i', 'i', 'the', 'that', 'i', 'the', 'that', 

4m 32s (- 74m 45s) (4300 5%) 4.5182
['', 'i', 'i', 'i', 'i', 'that', 'to', 'i', 'it', 'was', 'in', 'a', '', '<EOS>']

4m 35s (- 74m 38s) (4350 5%) 4.7857
['', 'i', 'i', 'i', 'the', 'that', 'i', 'the', 'i', 'the', 'i', '<EOS>']

4m 38s (- 74m 29s) (4400 5%) 4.3294
['', 'i', 'do', 'not', 'sure', 'that', 'the', 'the', '', 'of', 'the', 'the', '', '<EOS>']

4m 41s (- 74m 28s) (4450 5%) 4.6180
['', 'i', 'do', 'i', 'is', 'i', 'of', 'that', 'the', 'a', 'is', 'a', 'the', '<EOS>']

4m 44s (- 74m 23s) (4500 6%) 4.5648
['', 'i', 'agree', 'did', 'you', 'know', 'that', 'is', 'a', 'a', 'a', 'about', 'the', '<EOS>']

4m 47s (- 74m 16s) (4550 6%) 4.7927
['', 'yeah', 'that', 't', 'know', 'what', 'is', 'a', 'good', 'to', 'do', 'you', '', '<EOS>']

4m 50s (- 74m 13s) (4600 6%) 4.7680
['', 'i', 'i', 'that', 'that', 'to', 'i', 'that', 'that', 'to', 'i', 'that', 'that', 'to', 'you', '<EOS>']

4m 54s (- 74m 10s) (4650 6%) 3.8601
['', 'i', 'that', 'that', 'that', 'the', 'that', 'that', '', 'that', 'the', '', '

8m 4s (- 69m 34s) (7800 10%) 4.4246
['', 'i', 'know', 'the', 'the', 'the', 'it', 'of', 'the', 'the', '<EOS>']

8m 7s (- 69m 32s) (7850 10%) 4.8036
['', 'i', 'guess', 'that', 'i', 'a', 'to', 'of', 'the', 'i', 'to', 'of', 'a', 'to', 'of', 'a', 'i', 'to', 'of', 'a', 'the', 'i', 'to', 'of', 'a', 'to', 'of', 'a', '<EOS>']

8m 11s (- 69m 34s) (7900 10%) 4.6581
['', 'that', 'is', 'the', 'a', 'think', 'of', 'of', 'the', 'a', 'that', 'of', 'a', 'it', 'of', 'the', 'a', '<EOS>']

8m 14s (- 69m 30s) (7950 10%) 4.4075
['', 'wow', 'i', 'that', 'is', 'a', 'in', 'of', '', '', '<EOS>']

8m 17s (- 69m 30s) (8000 10%) 4.5637
['', 'that', 'is', 'the', 'the', 'the', 'the', 'the', 'to', 'the', '', 'the', '<EOS>']

8m 21s (- 69m 27s) (8050 10%) 4.3680
['', 'i', 'know', 'that', 'i', 'of', 'the', '<EOS>']

8m 24s (- 69m 25s) (8100 10%) 4.3312
['', 'yeah', 'i', 'think', 'the', 'is', 'that', 'the', 'of', 'the', 'the', 'you', '', 'the', '<EOS>']

8m 27s (- 69m 19s) (8150 10%) 4.3208
['', 'i', 'that', 'that', 'the

11m 48s (- 66m 52s) (11250 15%) 3.9096
['', 'that', 'is', 'a', 'it', 'i', 'the', 'it', 'the', 'a', 'the', '<EOS>']

11m 51s (- 66m 48s) (11300 15%) 4.2237
['', 'that', 'is', 'interesting', 'that', 'would', 'be', 'a', 'to', 'the', 'to', 'the', '<EOS>']

11m 54s (- 66m 47s) (11350 15%) 4.7596
['', 'i', 'i', 'that', 'to', 'the', 'to', 'the', 'the', 'to', 'the', 'the', 'to', 'the', '<EOS>']

11m 57s (- 66m 45s) (11400 15%) 4.6950
['', 'i', 'would', 'be', 'in', 'the', '', 'of', 'the', '<EOS>']

12m 1s (- 66m 42s) (11450 15%) 4.1074
['', 'i', 'i', 't', 'that', 'i', 'to', '<EOS>']

12m 4s (- 66m 43s) (11500 15%) 4.0995
['', 'that', 'is', 'true', 'i', 'i', 'to', 'have', '', 'to', '<EOS>']

12m 8s (- 66m 40s) (11550 15%) 3.9885
['', 'i', 'is', 'sure', 'i', 'would', 'have', 'to', 'the', 'a', 'the', 'as', 'as', 'as', 'well', 'as', 'well', 'as', 'well', '', '<EOS>']

12m 11s (- 66m 37s) (11600 15%) 4.3676
['', 'yeah', 'i', 'have', 'a', 't', 'a', 'that', 'you', 'to', 'have', 'a', '<EOS>']

12m 14s 

15m 26s (- 63m 18s) (14700 19%) 4.1302
['', 'i', 'have', 'that', 'i', 'a', 'you', 'i', 'to', '<EOS>']

15m 29s (- 63m 15s) (14750 19%) 3.9756
['', 'i', 'like', 'to', 'you', 'i', '<EOS>']

15m 31s (- 63m 10s) (14800 19%) 3.7081
['', 'yeah', 'i', 'am', 'a', 'fan', 'of', 'i', 'i', 'i', 'i', '<EOS>']

15m 34s (- 63m 6s) (14850 19%) 4.0722
['', 'i', 'i', 'i', 'i', 'i', 'was', 'you', 'the', 'you', 'i', '', '<EOS>']

15m 38s (- 63m 4s) (14900 19%) 4.2772
['', 'i', 'i', 'is', 'i', 'is', 'that', 'i', 'the', 'the', 'of', 'the', '', 'the', '', '<EOS>']

15m 41s (- 63m 1s) (14950 19%) 4.6104
['', 'i', 'i', 'i', 'i', 'of', 'it', 'of', 'it', 'to', 'of', 'i', 'i', 'the', 'i', '', 'the', 'i', '', '<EOS>']

15m 44s (- 62m 58s) (15000 20%) 4.3715
['', 'yeah', 'i', 'is', 'is', 'that', 'to', 'of', 'the', 'you', 'that', 'i', 'to', 'the', 'the', '<EOS>']

15m 47s (- 62m 54s) (15050 20%) 4.1365
['', 'i', 'i', 'you', 'the', 'the', 'you', '', '<EOS>']

15m 51s (- 62m 52s) (15100 20%) 4.3506
['', 'yeah', 'i', '

19m 5s (- 60m 0s) (18100 24%) 4.2683
['', 'i', 'i', 'i', 'and', 'the', 'of', 'the', '<EOS>']

19m 8s (- 59m 56s) (18150 24%) 4.1540
['', 'i', 'i', 'i', 'i', 'of', 'of', 'i', 'the', 'a', 'the', '<EOS>']

19m 11s (- 59m 53s) (18200 24%) 3.8632
['', 'i', 'i', 'a', 'i', 'a', 'i', 'the', 'a', 'a', 'the', 'a', 'the', '', '<EOS>']

19m 14s (- 59m 51s) (18250 24%) 4.6530
['', 'i', 'i', 'i', 'i', 'i', 'i', 'of', 'i', 'a', 'it', '<EOS>']

19m 18s (- 59m 48s) (18300 24%) 4.5294
['', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', '', '<EOS>']

19m 21s (- 59m 45s) (18350 24%) 4.0883
['', 'i', 'i', 'i', 'i', 'i', 'the', 'i', 'the', 'i', 'the', '', 'the', '<EOS>']

19m 24s (- 59m 42s) (18400 24%) 4.1209
['', 'i', 'i', 'i', 'i', 'i', 'i', 'of', 'a', 'the', 'i', '', '<EOS>']

19m 27s (- 59m 39s) (18450 24%) 4.1917
['', 'i', 'i', 'i', 'i', 'of', 'you', 'of', 'the', 'i', '', 'the', '<EOS>']

19m 31s (- 59m 36s) (18500 24%) 4.3305
['', 'i', 'i', 'i', 'i', 'the', 'i', 'the', 'the', '', 'i', 'the', '', '<EOS>

22m 59s (- 56m 28s) (21700 28%) 4.1497
['', 'i', 'you', 'have', 'the', 'a', 'you', 'the', 'the', 'a', 'the', 'to', 'the', '', '', 'the', '<EOS>']

23m 3s (- 56m 27s) (21750 28%) 4.5874
['', 'i', 'you', 'the', 'a', 'the', 'i', 'the', 'the', 'a', 'the', 'i', '', '', '', '<EOS>']

23m 7s (- 56m 25s) (21800 29%) 4.4779
['', 'i', 'i', 'a', 'i', 'the', 'i', 'to', 'the', 'a', 'to', 'the', 'to', '<EOS>']



KeyboardInterrupt: 

In [37]:
evaluateRandomly(encoder1, attn_decoder1)

> wow that is a lot of knowledge for the future of cinematography .
= he was married to a white woman but the marriage was not legal until !
< he s ! good the not of not not not not not not not not have ! the university of the from not have of not have ! have ! of have ! have of have of have of have of have the of not not not not not not not have of not have of not have ! of have of have of have ! of have of have ! without of the white not not not not not not have the moon of not of not have of not not not not not not not have of not not have ! of the moon not have not not not not not not not not have of not have ! of have of have of have ! of the white not have of not have not have of not have of not have of not have ! of have the moon of not have ! have ! of have of have of have of have the moon of not have of not not not not not not not not not not not not have of not have not have of not not not have ! of the moon but he have the of not not not not have of not not have of not not h

< he i think we get will get get he get get get get get get get get get get get get get get get get get get he lol get get get he lol get get get get get get he get get get get get get get get get get get he up will get he get will get he get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get will get get get get he get get get get will get he get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get get g

< yeah i ! have ! of ! wear ! wear ! wear wear ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! have ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! have ! of ! of ! of ! have ! of ! of ! of ! of ! of ! of ! of ! of ! have ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! of ! wear ! o

Exercises
=========

-  Try with a different dataset

   -  Another language pair
   -  Human → Machine (e.g. IOT commands)
   -  Chat → Response
   -  Question → Answer

-  Replace the embeddings with pre-trained word embeddings such as word2vec or
   GloVe
-  Try with more layers, more hidden units, and more sentences. Compare
   the training time and results.
-  If you use a translation file where pairs have two of the same phrase
   (``I am test \t I am test``), you can use this as an autoencoder. Try
   this:

   -  Train as an autoencoder
   -  Save only the Encoder network
   -  Train a new Decoder for translation from there




In [56]:
output_words, attentions = evaluate(encoder1, attn_decoder1, 'what are you doing')

In [57]:
output_words

['i', 'don', 't', 'think', 'going', 'to', 'jazz', '?', '<EOS>']